In [50]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
num_clients = 100
num_selected = 5
num_rounds = 50
epochs = 5
batch_size = 32

# Creating decentralized datasets

traindata = datasets.MNIST('/data/mwj/dataset', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))
target_labels = torch.stack([traindata.targets == i for i in range(10)])
# torch.Size([10, 60000])
print(target_labels.shape)

l = []
target_labels_split = []
for i in range(5):
    #60000个数据,按照每600个分一组，总共103组
    l.append(torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0].size(0))
    target_labels_split += torch.split(torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0], int(60000 / num_clients))
print(len(target_labels_split))
print(l)

traindata_split = [torch.utils.data.Subset(traindata, tl) for tl in target_labels_split]
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

# 正常来说这里的长度都应该为100，因为一共100个客户端，但是由于每个类别的数量并不是6000，所以就会多出来3个，里面的样本肯定很少
print(len(traindata_split))
print(len(train_loader))

torch.Size([10, 60000])
103
[12665, 12089, 11263, 12183, 11800]
103
103


In [12]:
target_labels

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False]])

In [30]:
target_labels[(2 * 0):(2 * (0 + 1))]

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [33]:
target_labels[(2 * 0):(2 * (0 + 1))].sum(0).shape

torch.Size([60000])

In [24]:
target_labels[0][1]

tensor(True)

In [43]:
torch.where(target_labels[(2 * 0):(2 * (0 + 1))])[0].shape

torch.Size([12665])

In [44]:
torch.where(target_labels[(2 * 0):(2 * (0 + 1))].sum(0))[0].shape

torch.Size([12665])